<a href="https://colab.research.google.com/github/srv-sh/FakeNewsDetection/blob/main/FakeNewsDetection(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

 # Import data

In [ ]:


!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
# পারমিশন দেখে নিন
!chmod 600 /root/.kaggle/kaggle.json



In [ ]:
!kaggle datasets download -d cryptexcode/banfakenews

In [ ]:



!unzip banfakenews.zip



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd 
df1 = pd.read_csv("/content/Authentic-48K.csv")
df2 = pd.read_csv("/content/Fake-1K.csv")
df3 = pd.read_csv("/content/LabeledAuthentic-7K.csv")
df4 = pd.read_csv("/content/LabeledFake-1K.csv")
df5 = pd.read_csv("/content/drive/MyDrive/data/Fake1k.csv") # this data is from another dataset


In [ ]:

#df = pd.concat([df1[:2000], df2, df3[:1591],df4,df5])
df = pd.concat([df1, df2, df3,df4,df5])
df_size = df.shape

In [ ]:
df.columns

# data preprocessing

In [ ]:
#concating 2 fields from dataset
new_df = df.loc[: , ['content' , 'label']] 
new_df.shape

In [ ]:
#labels distribution
tmp =  new_df['label'].value_counts().sort_index()
import seaborn as sb
sb.countplot(x='label' , data = new_df , order = tmp.index)


In [ ]:
#Class weight Minority class will get higher priroty and majority class will get lower priroty
from sklearn.utils import class_weight
import numpy as np

class_weight = list(class_weight.compute_class_weight('balanced',np.unique(new_df['label']),new_df['label']))

In [ ]:
class_weight

In [ ]:
weights={}

In [ ]:
for index , weight in enumerate(class_weight):
  weights[index] = weight

In [ ]:
weights

In [ ]:
#shuffle the data
'''new_df = new_df.sample(frac = 1)
new_df.head(10)'''
import sklearn
df_shuffled=sklearn.utils.shuffle(new_df)
df_shuffled.head(20)

In [ ]:
list_content = df_shuffled['content'].tolist()
list_label = df_shuffled['label'].astype(int).tolist()

In [ ]:
#data cleaning using rex
import re
def rex(token):
    return re.sub("([A-Za-z0-9_:.]+|[\n]+|[\xa0]+|শেয়ার করুন)","",token)
list_content =[rex(token) for token in list_content]

In [ ]:
 from sklearn.model_selection import train_test_split
 training_sentences , testing_sentences, training_labels, testing_labels = train_test_split(list_content, list_label, test_size=0.20, random_state=42 , shuffle= True)

In [ ]:
len(training_sentences)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
puncList = ["।", "”", "“", "’"]
x= "".join(puncList)
filterString = x + '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n০১২৩৪৫৬৭৮৯'
tokenizer = Tokenizer(oov_token='<OOV>',filters=filterString)
tokenizer.fit_on_texts(training_sentences)

In [ ]:
list(tokenizer.word_index.items())[:20]

In [ ]:
vocab_size = len(tokenizer.word_index)+1

In [ ]:
vocab_size

In [ ]:
max_length = 1000
trunc_type = 'post'
padding_type = 'post'
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences= tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length , padding= padding_type , truncating= trunc_type)

In [ ]:


tokenizer.texts_to_sequences(['ভোকাবুলারি সাইজ ঠিক করে নিচ্ছি'])



In [ ]:


print(testing_padded[5])



In [ ]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

# Building the model

In [ ]:
embedding_dim = 50

In [ ]:
training_padded

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding , Bidirectional,LSTM ,Dense , Dropout
import keras
from keras import backend as K

In [ ]:
#LSTM model

In [ ]:
model = Sequential(
    [
     Embedding(input_dim=vocab_size,output_dim= embedding_dim , input_length=max_length),
     Bidirectional(LSTM(64,return_sequences=True)),
     Bidirectional(LSTM(32)),
     Dense(1, activation='sigmoid')
    ]
)

In [ ]:
# LSTM with regularization

In [ ]:
model_regularization = Sequential(
    [
     Embedding(input_dim=vocab_size,output_dim= embedding_dim , input_length=max_length),
     Bidirectional(LSTM(64,return_sequences=True)),
     Dropout(rate=.4),
     Bidirectional(LSTM(32)),
     Dropout(rate=.4),
     Dense(1, activation='sigmoid')
    ]
)

In [ ]:
# CNN 

In [ ]:
from keras.models import Sequential
from keras import layers
embedding_dim = 100

model_cnn = Sequential()
model_cnn.add(layers.Embedding(input_dim=vocab_size,output_dim= embedding_dim , input_length=max_length))
model_cnn.add(layers.Conv1D(128, 5, activation='relu'))
model_cnn.add(layers.GlobalMaxPooling1D())
model_cnn.add(layers.Dense(10, activation='relu'))
model_cnn.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
#model LSTM
model.compile(loss='binary_crossentropy' , optimizer= 'adam' , metrics= ['accuracy'])
#model with regularixation
model_regularization.compile(loss='binary_crossentropy' , optimizer= 'adam' , metrics= ['accuracy'])
#model CNN
model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_regularization.summary()

In [ ]:
model_cnn.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
#model LSTM
num_epochs = 10
history = model.fit(training_padded , training_labels , epochs= num_epochs,validation_data=(testing_padded , testing_labels), callbacks=[tensorboard_callback], verbose =2, class_weight= weights )
#history = tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model')

In [ ]:
# model regularization
num_epoch =10 
history_dropout = model_regularization.fit(training_padded , training_labels , epochs= num_epochs,validation_data=(testing_padded , testing_labels), callbacks=[tensorboard_callback], verbose =2, class_weight= weights )
#history_dropout = tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model_dropout')

In [ ]:
#model CNN
num_epoch = 10 
history_cnn = model_cnn.fit(training_padded , training_labels , epochs= num_epochs,validation_data=(testing_padded , testing_labels), callbacks=[tensorboard_callback], verbose =2, class_weight= weights )

In [ ]:
## অ্যাক্যুরেসি এবং লস, ম্যাটপ্লটলিব দিয়ে

import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

#plot for normal LSTM
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

#plot for LSTM with dropout
plot_graphs(history_dropout, "accuracy")
plot_graphs(history_dropout, "loss")

#plot for CNN
plot_graphs(history_cnn, "accuracy")
plot_graphs(history_cnn, "loss")



In [ ]:
# Define our custom loss function
def focal_loss(y_true, y_pred):

  gamma= 2.0
  alpha = 0.25
  pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
  pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
  return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))



In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

model.compile(loss='binary_crossentropy' , optimizer= 'adam' , metrics= METRICS)
model_regularization.compile(loss='binary_crossentropy' , optimizer= 'adam' , metrics= METRICS)
model_cnn.compile(loss='binary_crossentropy' , optimizer= 'adam' , metrics= METRICS)


In [ ]:
num_epochs = 10
weighted_history = model.fit(training_padded , training_labels , epochs= num_epochs,validation_data=(testing_padded , testing_labels), callbacks=[tensorboard_callback], verbose =2,class_weight = weights)

In [ ]:
num_epochs = 10
weighted_history_dropout = model_regularization.fit(training_padded , training_labels , epochs= num_epochs,validation_data=(testing_padded , testing_labels), callbacks=[tensorboard_callback], verbose =2,class_weight = weights)

In [ ]:
num_epochs = 10
weighted_history_cnn = model_regularization.fit(training_padded , training_labels , epochs= num_epochs,validation_data=(testing_padded , testing_labels), callbacks=[tensorboard_callback], verbose =2,class_weight = weights)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'auc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()


In [ ]:

def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))


In [ ]:
#plot for LSTM
plot_metrics(weighted_history)


In [ ]:
#plot for LSTM (with Dropout)
plot_metrics(weighted_history_dropout)

In [ ]:
plot_metrics(weighted_history_cnn)

In [ ]:
# for LSTM
test_predictions_baseline = model.predict(testing_padded, batch_size=1000)

baseline_results = model.evaluate(testing_padded , testing_labels,
                                  batch_size=1000, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(testing_labels, test_predictions_baseline)


In [ ]:
# for LSTM(dropout)
test_predictions_baseline = model_regularization.predict(testing_padded, batch_size=1000)

baseline_results = model_regularization.evaluate(testing_padded , testing_labels,
                                  batch_size=1000, verbose=0)
for name, value in zip(model_regularization.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(testing_labels, test_predictions_baseline)

In [ ]:
# for CNN
test_predictions_baseline = model_cnn.predict(testing_padded, batch_size=1000)

baseline_results = model_cnn.evaluate(testing_padded , testing_labels,
                                  batch_size=1000, verbose=0)
for name, value in zip(model_cnn.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(testing_labels, test_predictions_baseline)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir -p '/content/gdrive/My Drive/Colab Notebooks/saved_model/'

In [ ]:


model.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model')



In [ ]:
model_regularization.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model_dropout')

In [ ]:
# embedding save

weights = model.get_layer('embedding_2').get_weights()[0]
#shape should be (vocab_size , embedding_dim)
print(weights.shape)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_sentence(training_padded[0]))
print(training_sentences[2])
print(training_labels[2])